In [2]:
import os
import numpy as np, pandas as pd, xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, f1_score
import joblib

In [3]:
train_data = pd.read_csv(
    os.path.join(
        os.path.dirname(os.getcwd()),
        'data',
        'training',
        'final_anomaly_training.csv'
    )
)
train_data.describe()

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
count,5.000000e+05,5.000000e+05,5.000000e+05,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000
mean,7.620937e-01,1.327140e+00,5.115073e-01,-0.118658,0.350732,0.100730,-0.349274,0.087404
std,2.952638e+00,9.519952e+00,1.749233e+00,0.323386,0.477200,0.300971,0.476741,0.282426
min,-4.522426e-01,-3.267150e-01,-6.123311e-01,-1.000000,0.000000,0.000000,-1.000000,0.000000
25%,-2.770901e-01,-2.293651e-01,-3.214841e-01,0.000000,0.000000,0.000000,-1.000000,0.000000
50%,4.034820e-17,-1.820020e-17,3.426058e-17,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7.229099e-01,7.706349e-01,6.785159e-01,0.000000,1.000000,0.000000,0.000000,0.000000
max,2.629377e+02,3.885243e+03,1.646674e+02,0.000000,1.000000,1.000000,0.000000,1.000000


In [4]:
X_train = train_data.drop(columns=['fraud'])
y_train = train_data['fraud']

spw = (y_train == 0).sum() / max(1, (y_train == 1).sum())

In [5]:
model = xgb.XGBClassifier(
    objective="binary:logistic",
    eval_metric="aucpr",            # better for rare positives
    scale_pos_weight=spw,
    n_estimators=2000,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_weight=1,
    tree_method="hist",
    n_jobs=-1,
    random_state=42,
)

In [6]:
eval_data = pd.read_csv(
    os.path.join(
        os.path.dirname(os.getcwd()),
        'data',
        'testing',
        'final_heldout_testing.csv'
    )
)
eval_data.describe()


,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
count,250000.000000,250000.000000,250000.000000,250000.000000,250000.00000,250000.000000,250000.000000,250000.000000
mean,0.751245,1.322413,0.508681,-0.118036,0.34984,0.100684,-0.349688,0.087404
std,3.044706,7.214491,1.702321,0.322651,0.47692,0.300910,0.476872,0.282427
min,-0.453118,-0.326649,-0.612094,-1.000000,0.00000,0.000000,-1.000000,0.000000
25%,-0.277074,-0.229560,-0.322691,0.000000,0.00000,0.000000,-1.000000,0.000000
50%,-0.000638,0.002164,0.000024,0.000000,0.00000,0.000000,0.000000,0.000000
75%,0.708908,0.780644,0.676347,0.000000,1.00000,0.000000,0.000000,0.000000
max,482.571443,872.947532,103.155886,0.000000,1.00000,1.000000,0.000000,1.000000


In [7]:
X_val = eval_data.drop(columns=['fraud'])
y_val = eval_data['fraud']

model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=False
)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='aucpr', feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=1, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=2000,
              n_jobs=-1, num_parallel_tree=None, ...)

In [8]:
p_val = model.predict_proba(X_val)[:, 1]
prec, rec, thr = precision_recall_curve(y_val, p_val)
f1s = 2*prec*rec/(prec+rec+1e-12)
best_thr = float(thr[np.argmax(f1s[:-1])])
print("Best F1:", f1_score(y_val, p_val >= best_thr), " @ thr=", best_thr)

Best F1: 0.9887953264416603  @ thr= 0.7745236158370972


In [9]:
test_data = pd.read_csv(
    os.path.join(
        os.path.dirname(os.getcwd()),
        'data',
        'testing',
        'final_split_testing.csv'
    )
)
test_data.describe()

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
count,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000
mean,0.749996,1.321497,0.508218,-0.118504,0.350292,0.100288,-0.349556,0.087400
std,2.931037,7.339187,1.710100,0.323205,0.477062,0.300384,0.476831,0.282421
min,-0.452080,-0.326656,-0.613159,-1.000000,0.000000,0.000000,-1.000000,0.000000
25%,-0.277367,-0.229685,-0.323477,0.000000,0.000000,0.000000,-1.000000,0.000000
50%,-0.001399,0.000538,-0.000515,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.710764,0.772658,0.678479,0.000000,1.000000,0.000000,0.000000,0.000000
max,398.275579,892.868770,89.624772,0.000000,1.000000,1.000000,0.000000,1.000000


In [13]:
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report

X_test = test_data.drop(columns=['fraud'])
y_test = test_data['fraud']

p_test = model.predict_proba(X_test)[:, 1]
y_pred = (p_test >= best_thr).astype(int)

roc_auc = roc_auc_score(y_test, p_test)
cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred, zero_division=0, digits=4)

metrics_table = pd.DataFrame({
    'Metric': ['ROC AUC'],
    'Value': [roc_auc]
})

print("Confusion Matrix:\n", cm)
print("\nClassification Report:\n", report)
print("\nMetrics Table:\n", metrics_table)

Confusion Matrix:
 [[227875    275]
 [   182  21668]]

Classification Report:
               precision    recall  f1-score   support

         0.0     0.9992    0.9988    0.9990    228150
         1.0     0.9875    0.9917    0.9896     21850

    accuracy                         0.9982    250000
   macro avg     0.9933    0.9952    0.9943    250000
weighted avg     0.9982    0.9982    0.9982    250000


Metrics Table:
     Metric     Value
0  ROC AUC  0.999971


In [12]:
dirpath = os.path.join(os.path.dirname(os.getcwd()), "results")
filepath = os.path.join(dirpath, "xgboost_model.joblib")
if not os.path.exists(dirpath):
    os.makedirs(dirpath)
if os.path.exists(filepath):
    os.remove(filepath)
joblib.dump(model, filepath)

['c:\\Users\\abudi\\OneDrive\\Documents\\Uni Work\\Capstone Project\\fraud-detection-capstone\\results\\xgboost_model.joblib']